In [1]:
import pandas as pd
from difflib import SequenceMatcher

In [2]:
# sample
df = pd.read_csv('totalset.csv', compression = 'gzip')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df= df[~df.identifier.str.contains("_reocr")].reset_index(drop = True)
df_1 = df[:34808]
df_2 = df[34808:36375]
df_3 = df[36375:40375-2000]
df_4 = df[40375-2000:40594-2000]
df_5 = df[40594-2000:]

selections = []
n = 5 #samples per dataset

for dataset in [df_1, df_2, df_3, df_4, df_5]:
    docs = dataset.sample(n)
    docs['dataset'] = f'{dataset}'
    selections.append(docs)

#sample = pd.concat(selections)

characters = 1000

for i in range(len(selections)):
    selections[i] = selections[i].reset_index(drop=True)
    for j in range(len(selections[i])):
        if len(selections[i]['gt text'][i]) > characters:
            first_page = selections[i]['gt text'][j][0:characters].split(".")
            first_page = '.'.join(first_page[0:-1])
            selections[i]['gt text'][j] = first_page
        if len(selections[i]['ocr text'][i]) > characters:
            first_page = selections[i]['ocr text'][j][0:characters].split(".")
            first_page = '.'.join(first_page[0:-1])
            selections[i]['ocr text'][j] = first_page

C:\Users\Gebruiker\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Gebruiker\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [3]:
# train = 80% of each dataset
# test = 20% of each dataset
import numpy as np

train_list = []
test_list = []

for i in range(len(selections)):
    a,b,c,d,e = np.array_split(selections[i], 5)
    train_list.append(pd.concat([a,b,c,d], axis=1).reset_index())
    test_list.append(e)

In [4]:
# matching sentences
def fuzzy_matching(GT, OCR): # create files with matched sentences
    print(GT[0:10])
    
    # split GT and OCR-output of each row into sentences (based on full stops).
    #gt_sentences = row['gt text'].split(".")
    #ocr_sentences = row['ocr text'].split(".")
    gt_sentences = GT.split('.')
    ocr_sentences = OCR.split('.')
    
    match_list = []

    for sentence in gt_sentences:
        for match in ocr_sentences:
            if SequenceMatcher(None, sentence, match).ratio() > 0.75:
                match_list.extend([[sentence, match]])
                
    matched_gt_sentences = []
    matched_ocr_sentences = []

    for sentence in match_list:
        if len(sentence[0]) or len(sentence[1]) > 0:
            matched_gt_sentences.append(sentence[0])
            matched_ocr_sentences.append(sentence[1])
    
    return matched_gt_sentences, matched_ocr_sentences


GT_matched_docs_datasets = []
OCR_matched_docs_datasets = []

for i in range(len(selections)):
    selections[i].reset_index(drop = True)
    selections[i]['fuzzy matched gt'] = ''
    selections[i]['fuzzy matched ocr'] = ''

    # match sentences GT and OCR
    GT_matched_docs_list = []
    OCR_matched_docs_list = []


    for j in range(len(selections[i])):
        matched_gt_sentences, matched_ocr_sentences = fuzzy_matching(selections[i]['gt text'][j][:100], selections[i]['ocr text'][j][:100])
        GT_matched_docs_list.append(matched_gt_sentences)
        OCR_matched_docs_list.append(matched_ocr_sentences) 
        selections[i]['fuzzy matched gt'].iloc[j] = '.'.join(matched_gt_sentences)
        selections[i]['fuzzy matched ocr'].iloc[j] = '.'.join(matched_ocr_sentences)

    GT_matched_docs_datasets.append(GT_matched_docs_list)
    OCR_matched_docs_datasets.append(OCR_matched_docs_list)
    #print(len(GT_matched_docs_list))
        
#print(GT_matched_docs_list[0][:10])
#print(OCR_matched_docs_list[0][:10])
#print(GT_matched_docs_datasets[0])
#print(OCR_matched_docs_datasets[0])
GT_sentences_datasets = GT_matched_docs_datasets
OCR_sentences_datasets = OCR_matched_docs_datasets

C:\Users\Gebruiker\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


ENGELANT, 
ENGELANT, 
Wert beken
Duytslant 
Wt Antwerp
VOORREDE. 
F 2 (23) E
VERHANDELI
P 225 GODS
WANSTALTIG
LEEUWARDER
echter nie
Het Nieuws
Honderden 
BUREAU'S V
WOORDENBOE
Boeck, ey 
Boekbescho
[Jacobus H
HUIBERT en
RADIO 2 va
Seriewedst
RADIO 2. T
EERSTE Uit
RADIO I. E


In [5]:
# word alignment
def word_alignment(GT, OCR):
    shortest_list = min([GT, OCR], key=len)
    longest_list = max([GT, OCR], key=len)
    min_ratio = 0.65

    for i in range(len(longest_list)):
        match = False
        #i = -i-1
        # check if there is a direct match:
        try:
            if OCR[i] == '' and GT[i] == '':
                del OCR[i]
                del GT[i]
            if SequenceMatcher(None, GT[i], OCR[i]).ratio() >= min_ratio:
                match = True
                continue
            elif match == False:
                # check if there is a match with a word before in OCR:
                try:
                    for j in range(1,10):
                        if i-j >= 0:
                            if j >= 5:
                                if (GT[i] >= 5) or (OCR[i-j] >= 5):
                                    pass
                                else:
                                    continue
                            if SequenceMatcher(None, GT[i], OCR[i-j]).ratio() >= min_ratio:
                                if not (GT[i] == '' and OCR[i-j] == ''):
                                    for k in range(0,j):
                                        OCR.insert(i-j, '')
                                    match = True
                                    continue
                        if i+j <= len(OCR):
                            if j >= 5:
                                if (GT[i] >= 5) or (OCR[i+j] >= 5):
                                    pass
                                else:
                                    continue
                            if SequenceMatcher(None, GT[i], OCR[i+j]).ratio() >= min_ratio:
                                if not (GT[i] == '' and OCR[i+j] == ''):
                                    for k in range(0,j):
                                        GT.insert(i, '')
                                    match = True
                                    continue
                except:
                    pass
        except IndexError:
            min([GT, OCR], key=len).append('')

        if match == False:
            try:
                if SequenceMatcher(None, GT[i], OCR[i]).ratio() >= 0.30:
                    pass
            except:
                pass
            else:
                print('bad match', GT[i], OCR[i])
                print('before forwards match GT:', GT)
                print('before forwards bad match OCR:', OCR)
                if i-1 >= 0:
                    GT.insert(i, '')
                else:
                    GT.insert(0, '')
                if i+1 <= len(GT):
                    OCR.insert(i+1, '')
                else:
                    GT.append('')
                print('after forwards match GT:', GT)
                print('after forwards bad match OCR:', OCR)
                
                continue
    
    print('after forwards GT:', GT)
    print('after forwards OCR:', OCR)
    
    while len(GT) != len(OCR):
        min([GT, OCR], key=len).append('')
    
    if True:
        
        shortest_list = min([GT, OCR], key=len)
        longest_list = max([GT, OCR], key=len)

        for i in range(len(longest_list)):
            i = -i-1
            print('i:', i)
            try:

                if (OCR[i] == '') ^ (GT[i] == ''):

                    continue
                if OCR[i] == ' ' and GT[i] == '':
                    del OCR[i]
                    del GT[i]
                if SequenceMatcher(None, GT[i], OCR[i]).ratio() > 0.65:
     
                    continue
                else:
                    # check if there is a match with a word before in OCR:
                    try:
                        for j in range(1,len(GT)):
                            if j >= 10:
                                extra_ratio = 0.1
                            else:
                                extra_ratio = 0
                            if i-j >= -len(OCR):
                                if j >= 5:
                                    if (len(GT[i]) >= 5) or (len(OCR[i-j]) >=5):
                                        pass
                                    else:
                                        continue
                            if GT[i] == '' or OCR[i-j] == '':
                                continue
                            if SequenceMatcher(None, GT[i], OCR[i-j]).ratio() > 0.65 + extra_ratio:
                                for k in range(0,j):
                                    if i+k < 0:
                                        GT.insert(i+1, '')
                                    else: 
                                        GT.append('')
                                break
                            if i+j <= -1:
                                if j >= 5:
                                    if (GT[i] >= 5) or (OCR[i+j] >=5):
                                        pass
                                    else:
                                        continue
                            if GT[i] == '' or OCR[i+j] == '':
                                continue
                            if SequenceMatcher(None, GT[i], OCR[i+j]).ratio() > 0.65 + extra_ratio:
                                for k in range(0,j):
                                    GT.insert(i, '')

                                break
                    except:
                        pass

            except IndexError:
                min([GT, OCR], key=len).append('')

            # if there is no match:
            if SequenceMatcher(None, GT[i], OCR[i]).ratio() > 0.30:
                pass
            else:
                print('bad match', GT[i], OCR[i])
                print('before backwards bad match GT:',  GT)
                print('before backwards bad match OCR:',  OCR)
                if (OCR[i] == '') or (GT[i] == ''):
                    continue
                else:
                    GT.insert(i, '')
                    OCR.insert(i+1, '')
                print('after backwards bad match GT:',  GT)
                print('after backwards bad match OCR:',  OCR)
                
                
    print('after backwards GT:',  GT)
    print('after backwards OCR:',  OCR)
                    
    while len(GT) != len(OCR):
        min([GT, OCR], key=len).append('')


    for i in range(len(GT)-1):
        try:
            print(i)
            print(GT[i], OCR[i])
            if (GT[i] == '') and (OCR[i] == ''):
                del GT[i]
                del OCR[i]
        except:
            break
    
    print('end GT:',  GT)
    print('end OCR:',  OCR)
    return GT, OCR

# word alignment
aligned_datasets_GT = []
aligned_datasets_OCR = []

for i in range(len(GT_sentences_datasets)): # for each dataset
    aligned_dataset_GT = []
    aligned_dataset_OCR = []
    for j in range(len(GT_sentences_datasets[i])): # for each document
        aligned_document_GT = []
        aligned_document_OCR = []
        for k in range(len(GT_sentences_datasets[i][j])): # for each sentence
            print('GT_sentences_datasets[i][j][k]:', GT_sentences_datasets[i][j][k])
            aligned_GT, aligned_OCR = word_alignment(GT_sentences_datasets[i][j][k].split(' '), OCR_sentences_datasets[i][j][k].split(' '))
            aligned_document_GT.append(aligned_GT)
            aligned_document_OCR.append(aligned_OCR)
        aligned_dataset_GT.append(aligned_document_GT)
        aligned_dataset_OCR.append(aligned_document_OCR)
    aligned_datasets_GT.append(aligned_dataset_GT)
    aligned_datasets_OCR.append(aligned_dataset_OCR)


GT_sentences_datasets[i][j][k]: ENGELANT, &amp;c
bad match &amp;c &c
before forwards match GT: ['ENGELANT,', '&amp;c']
before forwards bad match OCR: ['ENGELANT,', '&c']
after forwards match GT: ['ENGELANT,', '', '&amp;c']
after forwards bad match OCR: ['ENGELANT,', '&c', '']
after forwards GT: ['ENGELANT,', '', '&amp;c']
after forwards OCR: ['ENGELANT,', '&c', '']
i: -1
i: -2
i: -3
after backwards GT: ['ENGELANT,', '', '&amp;c']
after backwards OCR: ['ENGELANT,', '&c', '']
0
ENGELANT, ENGELANT,
1
 &c
end GT: ['ENGELANT,', '', '&amp;c']
end OCR: ['ENGELANT,', '&c', '']
GT_sentences_datasets[i][j][k]: ENGELANT, &amp;c
bad match &amp;c &c
before forwards match GT: ['ENGELANT,', '&amp;c']
before forwards bad match OCR: ['ENGELANT,', '&c']
after forwards match GT: ['ENGELANT,', '', '&amp;c']
after forwards bad match OCR: ['ENGELANT,', '&c', '']
after forwards GT: ['ENGELANT,', '', '&amp;c']
after forwards OCR: ['ENGELANT,', '&c', '']
i: -1
i: -2
i: -3
after backwards GT: ['ENGELANT,', '', 

In [6]:
def split_datasets_once(datasets):
    
    train_list = []
    test_list = []

    for i in range(len(datasets)):
        a,b,c,d,e = np.array_split(datasets[i], 5)
        train = np.concatenate([a,b,c,d], axis=0).tolist()
        test = e.tolist()
        train_list.append(train)
        test_list.append(test)

    #print('train_list:', train_list)
    #print('test_list:', test_list)
    return train_list, test_list

# train and test for aligned GT
train_list_GT_aligned, test_list_GT_aligned = split_datasets_once(aligned_datasets_GT)
# train and test for aligned OCR
train_list_OCR_aligned, test_list_OCR_aligned = split_datasets_once(aligned_datasets_OCR)
# train and test for sentences GT
train_list_GT_org, test_list_GT_org = split_datasets_once(GT_sentences_datasets)
# train and test for sentences OCR
train_list_OCR_org, test_list_OCR_org = split_datasets_once(OCR_sentences_datasets)

In [7]:
# put all sentences all documents train GT into one list
docs = [item for sublist in train_list_GT_org for item in sublist]
sentences_GT = [item for sublist in docs for item in sublist]

In [8]:
print(sentences_GT)

['ENGELANT, &amp;c', 'ENGELANT, &amp;c', ' Vrydag na de middag vergaderde den Geheymen Raed en klaegde', 'Wert bekent gemaeckt wegens den Heer Rentmeester Generael van de Graeffelijcke Domeynen van Zeelandt', "Duytslant en d\\'aengrensende Rijcken", ' XV zulks geraden vinden, zal be\ueba3oten wor‑ den tot het geeven van een Acces\ueba2t, aan den ſchr', 'F 2 (23) En in den avond van dien zelfden 18 January, in de Scheepskooy liggen- de, schryft hy aan M', 'VERHANDELINGEN EN WAARNEEMINGEN, OVER VERSCHEIDENE HEELKUNDIGE ONDERWERPEN, DOOR D', 'P 225 GODSDIENST EN TAAL', ' mers, schoone Rivieren, verschillende vrugtbaar- heid, bosschen en metaale', 'LEEUWARDER COURANT 208e JAARGANG UITGEEFSTER: Stichting FRIESE PERS DIRECTIE: S', ' Jonker en L', ' R', ' Sta', 'echter niet gelukte', ' Ook met achteruit werken kwam het schip niet vlot', ' Daar het ebtij was, werden v', 'Het Nieuwsblad voor Sumatra @OOFDRED', ' G', 'WOORDENBOEK VOOR VRIJMETSELAREN', ' ALGEMEEN WIJSGEERIG, GESCHIEDKUNDIG EN BIO

In [9]:
import gensim
from gensim.models import Word2Vec 
from gensim.models import KeyedVectors
import pandas as pd
from nltk.tokenize import RegexpTokenizer

# lowercase
#train_text = sentences_GT.split('.')
train_text = sentences_GT
train_text = [sentence.lower() for sentence in train_text ]
import re
train_text = [re.sub(r'[^\w\s]','',sentence) for sentence in train_text ]

# tokenize
tokenizer = RegexpTokenizer(r'\w+')
#test_text_sentences_tokenized = [w.lower() for w in sentences]
train_text = [tokenizer.tokenize(i) for i in train_text]

total_examples = len(train_text)

In [73]:
#hyperparameters
window = 5
top_n_detection = 100
top_n_correction = 100
ranking_method = 'sort' # choose 'sort' or 'score'

In [11]:
padding = int((window-1)/2)
padded_train = train_text
for i in range(0,padding):
    [sentence.insert(0, 'openingpadding') for sentence in padded_train]
    [sentence.append('closingpadding') for sentence in padded_train]

print(padded_train)

[['openingpadding', 'openingpadding', 'engelant', 'ampc', 'closingpadding', 'closingpadding'], ['openingpadding', 'openingpadding', 'engelant', 'ampc', 'closingpadding', 'closingpadding'], ['openingpadding', 'openingpadding', 'vrydag', 'na', 'de', 'middag', 'vergaderde', 'den', 'geheymen', 'raed', 'en', 'klaegde', 'closingpadding', 'closingpadding'], ['openingpadding', 'openingpadding', 'wert', 'bekent', 'gemaeckt', 'wegens', 'den', 'heer', 'rentmeester', 'generael', 'van', 'de', 'graeffelijcke', 'domeynen', 'van', 'zeelandt', 'closingpadding', 'closingpadding'], ['openingpadding', 'openingpadding', 'duytslant', 'en', 'daengrensende', 'rijcken', 'closingpadding', 'closingpadding'], ['openingpadding', 'openingpadding', 'xv', 'zulks', 'geraden', 'vinden', 'zal', 'beoten', 'wor', 'den', 'tot', 'het', 'geeven', 'van', 'een', 'accest', 'aan', 'den', 'ſchr', 'closingpadding', 'closingpadding'], ['openingpadding', 'openingpadding', 'f', '2', '23', 'en', 'in', 'den', 'avond', 'van', 'dien', 'z

In [12]:
from gensim.models import KeyedVectors

sentences = padded_train

model_w2v = Word2Vec(size=160, min_count=1, window=window)
print('1')
model_w2v.build_vocab(sentences)
print('2')
total_examples = model_w2v.corpus_count
print('3')
model = KeyedVectors.load_word2vec_format(r"combined-160.txt", binary=False)
print('4')
model_w2v.build_vocab([list(model.vocab.keys())], update=True)
print('5')
model_w2v.intersect_word2vec_format(r"combined-160.txt", binary=False, lockf=1.0)
print('6')
model_w2v.train(sentences, total_examples=total_examples, epochs=model_w2v.iter)
print('7')

1
2
3
4
5
6
7


C:\Users\Gebruiker\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


In [13]:
#test_list_GT_aligned
#test_list_OCR_aligned

In [14]:
print(test_list_GT_aligned)
print(test_list_OCR_aligned)

[[[['Wt', 'Antwerpen', 'den', '21', 'dito']]], [[]], [[["BUREAU'S", 'VOOR', 'REDACTIE', 'EN', 'ADMINISTRATIE:', 'WILHELMINASTRAAT', '23', '-', 'ZAANDAM', 'REDACTIE', '', '3633', '', 'TELEFOON', 'ADMINI']]], [[['HUIBERT', '', 'en', 'KLAARTJE'], ['HUIBERT', '', 'en', 'KLAARTJE'], ['HUIBERT', '', 'en', 'KLAARTJE'], ['HUIBERT', '', '', 'en', 'KLAARTJE'], ['EENE', 'VERTELLING', ''], ['DOOR', 'B', ''], ['TER', 'HAAR', ''], ['Met', 'vijftig', 'Houtgravure', '']]], [[['', 'RADIO', 'I']]]]
[[[['Wt', 'Antwerpen', 'den', '21', 'dito']]], [[]], [[['BUREAUS', 'VOOR', 'REDACTIE', 'EN', 'ADMINISTRATIE:', 'WILHELMINASTRAAT', '23', '-', 'ZAANDAM', 'REDACTIE', '3', '63', '3', 'TELEFOON', 'ADMIN']]], [[['ITIUIBERT', 'EN', '', 'KLAARTJE'], ['HUIBERT', 'Ex', '', 'KLAARTJE'], ['ITIUIBERT', 'EN', '', 'KLAARTJE'], ['HUIBERT', 'Ex', '', '', 'KLAARTJE'], ['EENE', 'VERTELLING', ''], ['DOOR', 'B', ''], ['TER', 'HAAR', ''], ['Met', 'vijftig', 'Houtgravu', '']]], [[['b"', 'RADIO', 'I']]]]


In [15]:
fake_test_list_GT_aligned = """Een koekenpan of kortweg pan is een platte pan met een lang handvat.
De pan ontleent zijn naam aan het feit dat in zo'n pan pannenkoeken worden gebakken. Ook ander voedsel, zoals vlees, wordt in een koekenpan gebraden."""
fake_test_list_OCR_aligned = """Een hoekenpan of hortweg pan is een hlatte pan met een hang handvat.
De pan ontleent zijn naam haan het heit dat in ho'n pan pannenkoeken horden gebakken. Ook hnder voedsel, zoals vlees, word in een koekenpan gebraden."""
fake_test_list_GT_aligned = fake_test_list_GT_aligned.split('.')
fake_test_list_OCR_aligned = fake_test_list_OCR_aligned.split('.')

token_GT = fake_test_list_GT_aligned 
token_GT = [sentence.lower() for sentence in token_GT ]
token_GT = [re.sub(r'[^\w\s]','',sentence) for sentence in token_GT ]
tokenizer = RegexpTokenizer(r'\w+')
fake_test_list_GT_aligned  = [tokenizer.tokenize(i) for i in token_GT]

token_OCR = fake_test_list_OCR_aligned 
token_OCR = [sentence.lower() for sentence in token_OCR ]
token_OCR = [re.sub(r'[^\w\s]','',sentence) for sentence in token_OCR ]
tokenizer = RegexpTokenizer(r'\w+')
fake_test_list_OCR_aligned = [tokenizer.tokenize(i) for i in token_OCR]


In [16]:
padding = int((window-1)/2)
padded_test_GT = fake_test_list_GT_aligned
for i in range(0,padding):
    [sentence.insert(0, 'openingpadding') for sentence in padded_test_GT]
    [sentence.append('closingpadding') for sentence in padded_test_GT]

print(padded_test_GT)


padded_test_OCR = fake_test_list_OCR_aligned
for i in range(0,padding):
    [sentence.insert(0, 'openingpadding') for sentence in padded_test_OCR]
    [sentence.append('closingpadding') for sentence in padded_test_OCR]

print(padded_test_OCR)


[['openingpadding', 'openingpadding', 'een', 'koekenpan', 'of', 'kortweg', 'pan', 'is', 'een', 'platte', 'pan', 'met', 'een', 'lang', 'handvat', 'closingpadding', 'closingpadding'], ['openingpadding', 'openingpadding', 'de', 'pan', 'ontleent', 'zijn', 'naam', 'aan', 'het', 'feit', 'dat', 'in', 'zon', 'pan', 'pannenkoeken', 'worden', 'gebakken', 'closingpadding', 'closingpadding'], ['openingpadding', 'openingpadding', 'ook', 'ander', 'voedsel', 'zoals', 'vlees', 'wordt', 'in', 'een', 'koekenpan', 'gebraden', 'closingpadding', 'closingpadding'], ['openingpadding', 'openingpadding', 'closingpadding', 'closingpadding']]
[['openingpadding', 'openingpadding', 'een', 'hoekenpan', 'of', 'hortweg', 'pan', 'is', 'een', 'hlatte', 'pan', 'met', 'een', 'hang', 'handvat', 'closingpadding', 'closingpadding'], ['openingpadding', 'openingpadding', 'de', 'pan', 'ontleent', 'zijn', 'naam', 'haan', 'het', 'heit', 'dat', 'in', 'hon', 'pan', 'pannenkoeken', 'horden', 'gebakken', 'closingpadding', 'closingpa

In [17]:
def detection_evaluation(actual_error, predicted_error, TP, FP, TN, FN):
    if actual_error == True:
        if predicted_error == True: # TP
            TP += 1
        if predicted_error == False: # FN
            FN += 1
    if actual_error == False:
        if predicted_error == True: # FP
            FP += 1
        if predicted_error == False: # TN
            TN += 1
    return TP, FP, TN, FN

# count TP, FP, TN, FN
TP = 0 # when model says it's an error (predicted_error == True) and it's actually an error (actual_error == True)
FP = 0 # when model says it's an error (predicted_error == True) and it's not actually an error (actual_error == False)
TN = 0 # when model says it's not an error (predicted_error == False) and it's actually not an error (actual_error == False)
FN = 0 # when model says it's not an error (predicted_error == False) and it's actually an error (actual_error == True)

In [22]:
# predict output word
for i in range(len(padded_test_OCR)):
    sentence = padded_test_OCR[i]
    for j in range(padding, len(sentence)-padding):
        print('word:', sentence[j])
        context_list = sentence[j-padding:j] + sentence[j+1:j+padding+1]
        predictions = []
        for prediction in model_w2v.predict_output_word(context_list, topn=top_n_detection):
            predictions.append(prediction[0])
        print(predictions[:10])
        if sentence[j] in predictions:
            predicted_error = False
        else: 
            predicted_error = True
        print('org:', padded_test_OCR[i][j], padded_test_GT[i][j])
        if padded_test_OCR[i][j] == padded_test_GT[i][j]:
            actual_error = False
        else:
            actual_error = True
        
        
        TP, FP, TN, FN = detection_evaluation(actual_error, predicted_error, TP, FP, TN, FN)
        
        print('context_list:', context_list)
        print('predicted_error:', predicted_error)
        print('actual_error:', actual_error)
        
        
        

word: een
['van', 'en', 'de', 'den', 'in', 'closingpadding', 'openingpadding', 'het', 'niet', 'te']
org: een een
context_list: ['openingpadding', 'openingpadding', 'hoekenpan', 'of']
predicted_error: False
actual_error: False
word: hoekenpan
['van', 'en', 'de', 'den', 'in', 'closingpadding', 'openingpadding', 'het', 'niet', 'te']
org: hoekenpan koekenpan
context_list: ['openingpadding', 'een', 'of', 'hortweg']
predicted_error: True
actual_error: True
word: of
['van', 'en', 'de', 'den', 'in', 'closingpadding', 'openingpadding', 'het', 'niet', 'aan']
org: of of
context_list: ['een', 'hoekenpan', 'hortweg', 'pan']
predicted_error: True
actual_error: False
word: hortweg
['van', 'en', 'de', 'den', 'closingpadding', 'in', 'openingpadding', 'het', 'niet', 'te']
org: hortweg kortweg
context_list: ['hoekenpan', 'of', 'pan', 'is']
predicted_error: True
actual_error: True
word: pan
['van', 'en', 'de', 'den', 'closingpadding', 'in', 'openingpadding', 'het', 'niet', 'te']
org: pan pan
context_list:

In [19]:
# calculate precision, recall, F1.
# ERROR DETECTION WORD2VEC
print('TP', TP)
print('FP', FP)
print('TN', TN)
print('FN', FN)

accuracy = (TP + TN)/(TP + TN + FP + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
F1 = 2*((precision*recall)/(precision+recall))

print('accuracy:', accuracy)
print('precision:', precision)
print('recall:', recall)
print('F1:', F1)

TP 10
FP 17
TN 11
FN 0
accuracy: 0.5526315789473685
precision: 0.37037037037037035
recall: 1.0
F1: 0.5405405405405406


In [71]:
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
stop_words = set(stopwords.words('dutch'))

def remove_stopwords(candidates, cosine, LD, stop_words):
    candidates_nostopwords = []
    cosine_nostopwords = []
    LD_nostopwords = []
    for i in range(len(candidates)):
        if candidates[i] not in stop_words:
            candidates_nostopwords.append(candidates[i])
            cosine_nostopwords.append(cosine[i])
            LD_nostopwords.append(LD[i])
    LD_nostopwords = np.array(LD_nostopwords)
    return candidates_nostopwords, cosine_nostopwords, LD_nostopwords

def correct_sorted(candidates, cosine, LD):
    paired_sorted = sorted(zip(LD,cosine,candidates),key = lambda x: (x[0],x[1]), reverse=True)
    LD,cosine,candidates = zip(*paired_sorted)
    print(candidates[:10])
    print(LD[:10])
    print(cosine[:10])
    correction = candidates[0]
    return correction
    


#def correct_calculated():
def correct_calculated(candidates, cosine, LD):
    inv_LD = 1 - LD
    cosine = np.array(cosine)
    cosine = np.interp(cosine, (cosine.min(), cosine.max()), (0, 1)).tolist()
    score = cosine / inv_LD
    #print(cosine)
    zipped_pairs = zip(score.tolist(), candidates)
    sorted_by_score = [x for _, x in sorted(zipped_pairs, reverse=True)]
    #print('sorted_by_score')
    #print(sorted_by_score)
    correction = sorted_by_score[0]
    return correction

right = 0
wrong = 0

# ERROR CORRECTION
for i in range(len(padded_test_OCR)): # for sentence
    sentence = padded_test_OCR[i]
    for j in range(padding, len(sentence)-padding): # for word in sentence
        if padded_test_OCR[i][j] != padded_test_GT[i][j]:
            print('word:', sentence[j])
            token = sentence[j]
            context_list = sentence[j-padding:j] + sentence[j+1:j+padding+1]
            candidates = []
            cosine = []
            for prediction in model_w2v.predict_output_word(context_list, topn=top_n_correction):
                candidates.append(prediction[0])
                cosine.append(prediction[1])
            LD = np.array([fuzz.ratio(word, token)/100 for word in candidates])
            #candidates, cosine, LD = remove_stopwords(candidates, cosine, LD, stop_words)
            if correction_method == 'sort':
                correction = correct_sorted(candidates, cosine, LD)
            elif correction_method == 'score':
                correction = correct_calculated(candidates, cosine, LD)
            if correction == padded_test_GT[i][j]:
                right += 1
            else:
                wrong += 1
                
# calculate
                
            

            

word: hoekenpan
('chelpan', 'shoenen', 'schekman', 'zoeperman', 'heptagoon', 'mazghoena', 'werken', 'bekent', 'ordena', 'olvena')
(0.62, 0.62, 0.59, 0.56, 0.56, 0.56, 0.53, 0.53, 0.53, 0.53)
(6.930143e-07, 6.930143e-07, 6.930143e-07, 6.930143e-07, 6.930143e-07, 6.930143e-07, 6.995302e-07, 6.976443e-07, 6.930143e-07, 6.930143e-07)
correction_sorted: chelpan
correction_calc: van
word: hortweg
('portner', 'popartwerken', 'bronsweg', 'esschoten', 'horecacao', 'fotoverwerking', 'themahotel', 'kortbestek', 'elmpterweg', 'portuenses')
(0.57, 0.53, 0.53, 0.5, 0.5, 0.48, 0.47, 0.47, 0.47, 0.47)
(6.9301467e-07, 6.9301467e-07, 6.9301467e-07, 6.9301467e-07, 6.9301467e-07, 6.9301467e-07, 6.9301467e-07, 6.9301467e-07, 6.9301467e-07, 6.9301467e-07)
correction_sorted: portner
correction_calc: en
word: hlatte
('attel', 'verplantte', 'atrytone', 'fshat', 'silikaten', 'holsatica', 'te', 'echter', 'themahotel', 'altier')
(0.73, 0.62, 0.57, 0.55, 0.53, 0.53, 0.5, 0.5, 0.5, 0.5)
(6.9301495e-07, 6.9301495e-0

In [25]:
from fuzzywuzzy import fuzz
target = 'koekenpan'
token = 'hoekenpan'
opties = ['wonen', 'koeken', 'pan', 'koekenpan', 'zoekenpan', 'aaaaa', 'oven', 'braadpan']
#cosine = [0.1, 0.5, 0.5, 0.7, 0.1, 0, 1, 0.8]
cosine = [1,5,5,9,0,1,1,8]
LD = [fuzz.ratio(word, token)/100 for word in opties]

ones = [1]*len(LD)
print(ones)

zipped_pairs = zip(LD, opties)
sorted_by_LD = [x for _, x in sorted(zipped_pairs)]
zipped_pairs = zip(cosine, sorted_by_LD)
sorted_by_LD_cosine = [x for _, x in sorted(zipped_pairs)]

#d = {'opties': opties, 'LD': LD, 'cosine': cosine}
#df = pd.DataFrame(data=d)
#df = df.sort_values(by=['LD'], ascending = False)
#df = df.sort_values(by=['cosine'], ascending = False)
#print(df)

print(sorted_by_LD_cosine)

d = {'opties': opties, 'LD': LD, 'cosine': cosine}
df3 = pd.DataFrame(data=d)
df3 = df3.sort_values(by=['LD','cosine'], ascending = False)
#df3 = df3.sort_values(by=['cosine'], ascending = False)
print(df3)


d = {'opties': opties, 'LD': LD, 'cosine': cosine}
df2 = pd.DataFrame(data=d)
df2['LD'] = 1 - df2['LD']
df2['score'] = df2['cosine']/(df2['LD'])
df2 = df2.sort_values(by=['score'], ascending = False)
print(df2)

[1, 1, 1, 1, 1, 1, 1, 1]
['pan', 'aaaaa', 'koeken', 'koekenpan', 'braadpan', 'wonen', 'zoekenpan', 'oven']
      opties    LD  cosine
3  koekenpan  0.89       9
4  zoekenpan  0.89       0
1     koeken  0.67       5
2        pan  0.50       5
6       oven  0.46       1
0      wonen  0.43       1
7   braadpan  0.35       8
5      aaaaa  0.14       1
      opties    LD  cosine      score
3  koekenpan  0.11       9  81.818182
1     koeken  0.33       5  15.151515
7   braadpan  0.65       8  12.307692
2        pan  0.50       5  10.000000
6       oven  0.54       1   1.851852
0      wonen  0.57       1   1.754386
5      aaaaa  0.86       1   1.162791
4  zoekenpan  0.11       0   0.000000


In [ ]:
print(LD)

In [26]:
import numpy as np
opties = ['wonen', 'koeken', 'pan', 'koekenpan', 'zoekenpan', 'aaaaa', 'oven', 'braadpan']
#cosine = [0.1, 0.5, 0.5, 0.7, 0.1, 0, 1, 0.8]
cosine = np.array([1,5,5,9,0,1,1,8])
LD = np.array([fuzz.ratio(word, token)/100 for word in opties])
inv_LD = 1 - LD
score = cosine / inv_LD
#sorted = np.sort(score, order=opties)
print(score)

zipped_pairs = zip(score.tolist(), opties)
sorted_by_score = [x for _, x in sorted(zipped_pairs, reverse=True)]

print(sorted_by_score)


[ 1.75438596 15.15151515 10.         81.81818182  0.          1.1627907
  1.85185185 12.30769231]
['koekenpan', 'koeken', 'braadpan', 'pan', 'oven', 'wonen', 'aaaaa', 'zoekenpan']


In [66]:


def correct_sorted(candidates, cosine, LD):
    #max_LD = LD.max()
    #zipped_pairs_LD = zip(LD.tolist(), candidates)
    #sorted_by_LD = [x for _, x in sorted(zipped_pairs_LD, reverse=True)]
    #print(sorted_by_LD)
    #max_LD_array = sorted_by_LD[sorted_by_LD != max_LD]
    #print(max_LD_array)
    #zipped_pairs_cos = zip(sorted_by_LD, candidates)
    #sorted_by_both = [x for _, x in sorted(zipped_pairs_cos, reverse=True)]
    
    #print('sorted_by_both')
    #print(sorted_by_both)
    paired_sorted = sorted(zip(LD,cosine,candidates),key = lambda x: (x[0],x[1]), reverse=True)
    LD,cosine,candidates = zip(*paired_sorted)
    #candidates = [v for i, v in sorted(enumerate(candidates), key=lambda i_v: (LD[i_v[0]], cosine[i_v[0]]), reverse=True)]
    print(candidates)
    print(LD)
    print(cosine)

    
def correct_calculated(candidates, cosine, LD):
    inv_LD = 1 - LD
    score = cosine / inv_LD
    zipped_pairs = zip(score.tolist(), candidates)
    sorted_by_score = [x for _, x in sorted(zipped_pairs, reverse=True)]
    print('sorted_by_score')
    print(sorted_by_score)
    correction = sorted_by_score[0]
    return correction


token = 'hoekenpan'
zipped_pairs = zip(score.tolist(), opties)
sorted_by_score = [x for _, x in sorted(zipped_pairs, reverse=True)]

opties = ['wonen', 'koeken', 'pan', 'koekenpan', 'zoekenpan', 'aaaaa', 'oven', 'braadpan']
cosine = np.array([1,5,5,9,0,1,1,8])
LD = np.array([fuzz.ratio(word, token)/100 for word in opties])

correct_sorted(opties, cosine, LD)
correction = correct_calculated(opties, cosine, LD)
print('correction:', correction)


('koekenpan', 'zoekenpan', 'koeken', 'pan', 'oven', 'wonen', 'braadpan', 'aaaaa')
(0.89, 0.89, 0.67, 0.5, 0.46, 0.43, 0.35, 0.14)
(9, 0, 5, 5, 1, 1, 8, 1)
sorted_by_score
['koekenpan', 'koeken', 'braadpan', 'pan', 'oven', 'wonen', 'aaaaa', 'zoekenpan']
correction: koekenpan


In [28]:
print(model_w2v.most_similar('haan'))

C:\Users\Gebruiker\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('siep', 0.7126805186271667), ('visser', 0.712116003036499), ('arie', 0.7109019756317139), ('wildbret', 0.6829932332038879), ('arend', 0.6817572116851807), ('koster', 0.681355893611908), ('houtzager', 0.6774497032165527), ('eikeboom', 0.6760517358779907), ('roeloff', 0.670612633228302), ('bok', 0.6673833727836609)]


In [39]:
a = np.array([0.07, 0.08, 0.09])
np.interp(a, (a.min(), a.max()), (0, 1))

array([0. , 0.5, 1. ])

In [50]:
l1 = [0.1,0.2,0.3,0.4,0.5]
l2 = [1,2,3,4,5]
l3 = ['a','b','c','d','e']
klinkers = ['a','e']

#for i in range(len(l3)):
#    print(l3)
#    print(i)
#    if l3[i] in klinkers:
#        del l1[i]
#        del l2[i]
 

l3_nostopwords = []
l2_nostopwords = []
l1_nostopwords = []
for i in range(len(l3)):
    if l3[i] not in klinkers:
        l3_nostopwords.append(l3[i])
        l2_nostopwords.append(l2[i])
        l1_nostopwords.append(l1[i])
    

        
print(l3_nostopwords)

['b', 'c', 'd']


In [8]:
import nltk
text = "marije woont in Gasjkdhalsdiouasdm. Ze woont samen met Kees?"
text = nltk.word_tokenize(text)
nltk.pos_tag(text)

[('marije', 'NN'),
 ('woont', 'NN'),
 ('in', 'IN'),
 ('Gasjkdhalsdiouasdm', 'NNP'),
 ('.', '.'),
 ('Ze', 'NNP'),
 ('woont', 'JJ'),
 ('samen', 'NNS'),
 ('met', 'VBD'),
 ('Kees', 'NNP'),
 ('?', '.')]